## Mean Shift 군집화

* Mean Shift는 KDE(Kernel Density Estimation)를 이용하여 데이터 포인트들이 데이퍼 분포가 높은 곳으로 이동하면서 군집화를 수행
* 별도의 군집화 개수를 지정하지 않으며 Mean Shift는 데이터 분포도에 기반하여 자동으로 군집화 개수를 정함

### Mean Shift 수행 절차

1. 개별 데이터의 특정 반경 내에 주변 데이터를 포함한 데이터 분포도 계산
2. 데이터 분포도가 높은 방향으로 중심점 이동
3. 중심점을 따라 해당 데이터 이동
4. 이동된 데이터의 특정 반경내에 다시 데이터 분포 계산 후 2,3 step을 반복
5. 가장 분포도가 높은 곳으로 이동하면 더 이상 해당 데이터는 움직이지 않고 수렴
6. 모든 데이터를 1~5까지 수행하면서 군집 중심점을 찾음

* 특정 데이터가 반경내의 데이터 분포 확률 밀도가 가장 높은 곳으로 이동 할 때 주변 데이터들과의 거리값을 Kernel 함수 값으로 입력 한 뒤 그 반환값을 현재 위치에서 Update하면서 이동

### KDE(Kernel Density Estimation)

* KDE는 커널(Kernel)함수를 통해 어떤 변수의 확률밀도 함수를 추정하는 방식. 관측된 데이터 각각에 커널 함수를 적용한 값을 모두 더한 뒤 데이터 건수로 나누어서 확률 밀도 함수를 추정.
* 확률밀도함수(PDF : Probability Density Function) : 확률 변수의 분포를 나타내는 함수. 대표적으로 정규 분포, 감마 분포, t-분포 등이 있음
* 확률밀도 함수를 알게 되면 특정 변수가 어떤 값을 갖게 될지의 확률을 알게 됨을 의미. 즉 확률밀도 함수를 통해 변수의 특성(예를 들어 정규 분포의 경우 평균, 분산), 확률 분포 등 변수의 많은 요소를 알 수 있게 됨

### 확률 밀도 추정 방법

* 모수적(Parametric) 추정 : 데이터가 특정 데이터 분포(예를 들어 가우시안 분포)를 따른 다는 가정하에 데이터 분포를 찾는 방법. Gaussian Mixture 등이 있음
* 비모수적(Non-Parametric) 추정 : 데이터가 특정 분포를 따르지 않는다는 가정 하에서 밀도를 추정, 관측된 데이터 만으로 확률 밀도를 찾는 방법으로 대표적으로 KDE가 있음

### 비모수적 밀도 추정 - 히스토그램(Histogram)

* 히스토그램 밀도 추정의 문제점
    * Bin의 경계에서 불연속성이 나타남
    * Bin의 크기에 따라 히스토그램이 달라짐

### 비모수적 밀도 추정 - KDE

* KDE는 개별 관측 데이터들에 커널함수를 적용한 뒤, 커널함수들의 적용값을 모두 합한 뒤에 개별 관측 데이터의 건수로 나누어서 확률밀도 함수를 추정하는 방식임. 커널함수로는 대표적으로 가우시안 분포함수가 사용됨.

### KDE와 가우시안 커널함수

* KDE는 아래와 같은 커널함수 식으로 표현됨. 이때 K는 커널함수, x는 random variable, xi는 관측값, h는 bandwidth

* $ KDE = \frac{1}{n} \sum_{i=1}^{n}K_h(x-x_i) = \frac{1}{nh} \sum_{i=1}^{n}K(\frac{x-x_i}{h}) $

* 대표적인 커널함수는 가우시안 분포임. $ f(x| \mu, \sigma^2) = \frac{1}{ \sqrt{2\pi\sigma^2} }e^-\frac{(x-\mu)^2}{2\sigma^2} $

* 가우시안 커널함수를 적용한 KDE는 아래와 같음. 이 경우 관측값 xi는 평균, bandwidth h는 표준편차와 동일.
    * $ KDE = \frac{1}{nh} \sum_{i=1}^{n}\frac{1}{\sqrt{2}\pi h}e^(-\frac{1}{2}(\frac{x-x_i}{h})^2 ) $
    * 가우시안 커널함수를 적용할 경우 최적의 bandwidth는 아래와 같습니다.
        * $ h=(\frac{4\sigma^5}{3n})^\frac{1}{5} \approx 1.06\sigma n^-\frac{1}{5} $

### Bandwidth에 따른 KDE의 변화

* 작은 h값은 좁고 spike한 KDE로 변동성이 큰 확률밀도함수를 추정(오버피팅)
* 큰 h값은 과도하게 Smoothing된 KDE로 단순화된 확률밀도함수를 추정(언더피팅)

* Mean Shift는 Bandwidth가 클수록 적은 수의 클러스터링 중심점을, Bandwidth가 작을수록 많은 수의 클러스터링 중심점을 가지게 됨, 또한 Mean Shift는 군집의 개수를 지정하지 않으며, 오직 Bandwidth의 크기에 따라 군집화를 수행.

### 사이킷런 Mean Shift

* 사이킷런은 Mean Shift 군집화를 위해 MeanShift 클래스를 제공
* MeanShift 클래스의 가장 중요한 초기화 파라미터는 bandwidth이며 해당 파라미터는 밀도 중심으로 이동 할때 사용되는 커널 함수의 bandwidth임. 이 bandwidth를 어떻게 설정하느냐에 따라 군집화 성능이 달라짐.
* 최적의 bandwidth 계산을 위해 사이킷런은 estimate_bandwidth()함수를 제공